In [3]:
# Cálcula el porcentaje de error entre dos series de maáximos anuales.

import pandas as pd
import xarray as xr

from scipy import stats

from matplotlib import pyplot as plt
import holoviews as hv
import geoviews as gv
from cartopy import feature as cf

cf_estados = cf.NaturalEarthFeature( category = "cultural", scale = "50m",
    name = "admin_1_states_provinces_lines", facecolor = "none" )
# Get convert to gv.Feature and extract geometries
gv_estados = gv.Feature(cf_estados).geoms().opts(
    line_color = "black", line_width = 0.8 )

In [9]:
i = 0
j = 2

name = [ "CHIRPS_comp", "CHIRPS_megalopolis", "prec_hist_comp_CHIRPS",
    "prec_hist_comp_estaciones", "prec_hist_hist" ]

path_orig = "../results/" + name[i] + "/" + name[i] + "_tretorno.nc" 
path_comp = "../results/" + name[j] + "/" + name[j] + "_tretorno.nc" 

ds_orig = xr.open_dataset(path_orig).drop( ["AÑO", "PROBABILIDAD"] )
ds_comp = xr.open_dataset(path_comp).drop( ["AÑO", "PROBABILIDAD"] )

In [3]:
# Error total.
error = ( ( ds_comp - ds_orig ) / ds_orig ).rename( {"INTENSIDAD": "P_ERROR"} )
error.to_netcdf("../results/comp_hist_CHIRPS.nc")

tot = [ error["P_ERROR"].to_numpy().min(), error["P_ERROR"].to_numpy().mean(),
        error["P_ERROR"].to_numpy().max(), error["P_ERROR"].to_numpy().std() ]

print(f"Errores entre {name[i]} y {name[j]}")
print(f"Mínimo             : { tot[0] * 100 :7.2f}%")
print(f"Media              : { tot[1] * 100 :7.2f}%")
print(f"Máximo             : { tot[2] * 100 :7.2f}%")
print(f"Desviación estándar: { tot[3] * 100 :7.2f}%")

In [ ]:
# Error por tiempo de retorno.
tretorno = error.mean( ["LONGITUD", "LATITUD"]
    ).min( "DURACION" ).to_dataframe()
tretorno.columns = ["MINIMO"]
tretorno["MEDIA"] = error.mean( ["LONGITUD", "LATITUD"]
    ).mean( "DURACION" ).to_dataframe()
tretorno["MAXIMO"] = error.mean( ["LONGITUD", "LATITUD"]
    ).max( "DURACION" ).to_dataframe()
tretorno["DESVIACION_ESTANDAR"] = error.mean( ["LONGITUD", "LATITUD"]
    ).std( "DURACION" ).to_dataframe()

tretorno

,Mínimo,Media,Máximo,Desviación estándar
TIEMPO_RETORNO,,,,
1.032258,-1.0,-1.0,-1.0,0.0
1.066667,-1.0,-1.0,-1.0,0.0
1.103448,-1.0,-1.0,-1.0,0.0
1.142857,-1.0,-1.0,-1.0,0.0
1.185185,-1.0,-1.0,-1.0,0.0
1.230769,-1.0,-1.0,-1.0,0.0
1.280000,-1.0,-1.0,-1.0,0.0
1.333333,-1.0,-1.0,-1.0,0.0
1.391304,-1.0,-1.0,-1.0,0.0


In [ ]:
# Error por duración.
duracion = error.mean( ["LONGITUD", "LATITUD"]
    ).min( "TIEMPO_RETORNO" ).to_dataframe()
duracion.columns = ["MINIMO"]
duracion["MEDIA"] = error.mean( ["LONGITUD", "LATITUD"]
    ).mean( "TIEMPO_RETORNO" ).to_dataframe()
duracion["MAXIMO"] = error.mean( ["LONGITUD", "LATITUD"]
    ).max( "TIEMPO_RETORNO" ).to_dataframe()
duracion["DESVIACION_ESTANDAR"] = error.mean( ["LONGITUD", "LATITUD"]
    ).std( "TIEMPO_RETORNO" ).to_dataframe()

duracion

,MINIMO,MEDIA,MAXIMO,DESVIACION_ESTANDAR
DURACION,,,,
1,-1.0,-1.0,-1.0,0.0
2,-1.0,-1.0,-1.0,0.0
3,-1.0,-1.0,-1.0,0.0
4,-1.0,-1.0,-1.0,0.0
5,-1.0,-1.0,-1.0,0.0


In [ ]:
# Dsitribución espacial del error.

espacial = error.min( ["DURACION", "TIEMPO_RETORNO"]
    ).rename( {"P_ERROR": "MINIMO"} )
espacial["MEDIA"]  = error.mean( ["DURACION",
    "TIEMPO_RETORNO"] )[ "P_ERROR" ]
espacial["MAXIMO"] = error.max( ["DURACION",
    "TIEMPO_RETORNO"] )[ "P_ERROR" ]
espacial["DESVIACION_ESTANDAR"] = error.std( ["DURACION",
    "TIEMPO_RETORNO"] )[ "P_ERROR" ]

espacial.to_netcdf("../results/comp_hist_CHIRPS_espacial.nc")

espacial

<xarray.Dataset>
Dimensions:              (west_east: 62, south_north: 50)
Coordinates:
    LONGITUD             (west_east) float32 -100.4 -100.4 ... -97.43 -97.38
    LATITUD              (south_north) float32 18.17 18.22 18.27 ... 20.57 20.62
Dimensions without coordinates: west_east, south_north
Data variables:
    MINIMO               (south_north, west_east) float64 -1.0 -1.0 ... -1.0
    MEDIA                (south_north, west_east) float64 -1.0 -1.0 ... -1.0
    MAXIMO               (south_north, west_east) float64 -1.0 -1.0 ... -1.0
    DESVIACION_ESTANDAR  (south_north, west_east) float64 0.0 0.0 ... 0.0 0.0

In [ ]:
# Graficas espaciales del error.

mean_fc = gv.FilledContours( espacial["MINIMO"] ).opts( 
    title = "Media del porcentaje de error entre CHIRPS y WRF",
    cmap = "bwr", colorbar = True, alpha = 0.8, fontsize = {"title": 13} )
std_fc  = gv.FilledContours( espacial["DESVIACION_ESTANDAR"] ).opts(
    title = "Desviación estándar del porcentaje de error entre CHIRPS y WRF",
    cmap = "bwr", colorbar = True, alpha = 0.8, fontsize = {"title": 13})

mean_img = ( mean_fc * gv_estados ).opts(fontscale = 2.5)
std_img  = ( mean_fc * gv_estados ).opts(fontscale = 2.5)

gv.output( mean_img, size = 600 )
gv.output( std_img, size = 600 )